# Test book

In [1]:
from scipy.io import loadmat
import mat73
import os
import numpy as np
import pandas as pd
from GLM import glm


In [ ]:
data=loadmat('/Users/user/PycharmProjects/PacManMain/GLM/exampleData/PMD.mat')
data=pd.DataFrame(data['data'][0])
data = data.applymap(lambda x: [item[0] for item in x])
data = data.apply(pd.Series.explode).reset_index(drop=True)
data['rewardVal']=data.rewardVal.astype('category')
data=data[data['rewardVal']!=0]
X=data[['self_spd','prey_spd','rewardVal']]

X['prey_spd'] = pd.to_numeric(X['prey_spd'], errors='coerce')
X['self_spd'] = pd.to_numeric(X['self_spd'], errors='coerce')

y = data['n3']

#Make formulas
formulas=[]
formulas.append('y ~ (cr(self_spd,df=5)+ cr(prey_spd, df=5))*C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5) + cr(prey_spd, df=5)+C(rewardVal) -1')
formulas.append('y ~ cr(self_spd,df=5)*C(rewardVal)-1')
formulas.append('y ~ cr(prey_spd,df=5)*C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5)+C(rewardVal)-1')
formulas.append('y ~ cr(prey_spd,df=5)+C(rewardVal)-1')
formulas.append('y ~ cr(self_spd,df=5)')
formulas.append('y ~ cr(prey_spd,df=5)')
formulas.append('y ~C(rewardVal)-1')
formulas.append('y ~1')

mod=glm.PoissonGLM()
mod.add_data(X,y).make_preprocessor(formulas=formulas,metric='cv',l2reg=0.01).fit(params={'cv': 5, 'shuffleTime': False})


# x1 = np.linspace(mod.X['prey_spd'].min(), mod.X['prey_spd'].max(), 100)  # 100 points in the range of x1
# pred_data = pd.DataFrame({'prey_spd':x1})
levels = [1, 2,3,4,5] 
pred_data = pd.DataFrame({'rewardVal':np.tile(levels,1)})
mod.predict(pred_data)

        # x2_median = data['x2'].median()  # Median value for x2
        # levels = ['A', 'B', 'C']  # Levels for categorical variable


# 
# levels = ['A', 'B']  # Levels for categorical variable
# 
# # Create a DataFrame to hold the predictions for each level of the categorical variable
# pred_data = pd.DataFrame({
#     'x1':x1_median,
#     'x2': np.repeat(x2_range,len(levels)),
#     'cat_var': np.tile(levels, len(x2_range))
# })

Generate data

Make predicted set